In [ ]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

In [ ]:
## Load dataset
K = 4
N = 64
data_path = data_dir + "ncmm/squares_%d" % N

OB = torch.from_numpy(np.load(data_path + '/ob_%d.npy' % (N * K))).float()
MU = torch.from_numpy(np.load(data_path + '/mu_%d.npy' % (N * K))).float()
ANGLE = torch.from_numpy(np.load(data_path + '/angle_%d.npy' % (N * K))).float()
STATE = torch.from_numpy(np.load(data_path + '/state_%d.npy' % (N * K))).float()
## Hyper Parameters
NUM_DATASETS = OB.shape[0]

## Train Parameters
NUM_EPOCHS = 500

D = 2
SAMPLE_SIZE = 10
BATCH_SIZE = 20
NUM_BATCHES = int((NUM_DATASETS / BATCH_SIZE))

CUDA = torch.cuda.is_available()
DEVICE = 1
RECON_SIGMA = torch.ones(1) * 0.2
# lg2pi = torch.log(torch.ones(1) * 2 * math.pi)
PATH = 'dec-%dpts-%dsamples' % (N*K, SAMPLE_SIZE)
## MOdel Parameters
NUM_HIDDEN_LOCAL = 64
NUM_HIDDEN = 32
NUM_NSS = 8
LEARNING_RATE = 1e-3
Train_Params = (NUM_EPOCHS, K, D, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)

In [ ]:
from decoder_semi_v2 import *
dec_x = Dec_x(D, NUM_HIDDEN, RECON_SIGMA, CUDA, DEVICE)
if CUDA:
    with torch.cuda.device(DEVICE):
        dec_x.cuda()
optimizer =  torch.optim.Adam(list(dec_x.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))

In [ ]:
for epoch in range(NUM_EPOCHS):
    time_start = time.time()
    LOSS = 0.0
    indices = torch.randperm(NUM_DATASETS)
    for step in range(NUM_BATCHES):
        optimizer.zero_grad()
        batch_indices = indices[step*BATCH_SIZE : (step+1)*BATCH_SIZE]
        ob = OB[batch_indices]
        state = STATE[batch_indices]
        mu = MU[batch_indices]
        angle = ANGLE[batch_indices]
        
#         labels = state.argmax(-1).unsqueeze(-1).repeat(1, 1, D)
#         mu_copy = torch.gather(mu, 1, labels)
        ob_embedding = torch.cat((ob, state, angle), -1)

        ob_embedding = shuffler(ob_embedding).repeat(SAMPLE_SIZE, 1, 1, 1)
        mu = mu.repeat(SAMPLE_SIZE, 1, 1, 1)
        if CUDA:
            with torch.cuda.device(DEVICE):
                ob_embedding =ob_embedding.cuda()
                mu = mu.cuda()
        ob = ob_embedding[:, :, :, :2]
        state = ob_embedding[:, :, :, 2:-1]
        angle = ob_embedding[:, :, :, -1].unsqueeze(-1)
        p = dec_x(ob, state, angle, mu)
        loss =  - p['likelihood'].log_prob.sum()
        ## gradient step
        loss.backward()
        optimizer.step()  
        LOSS += loss
    time_end = time.time()
    print('epoch=%d, loss=%.4f (%ds)' % (epoch, LOSS / NUM_BATCHES, time_end - time_start))

In [ ]:
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

def plot_recon(recon, path, page_width, bound):
    B, N, D = recon.shape
    gs = gridspec.GridSpec(int(B / 5), 5)
    gs.update(left=0.0 , bottom=0.0, right=1.0, top=1.0, wspace=0, hspace=0)
    fig = plt.figure(figsize=(page_width,page_width*4/5))
    for b in range(B):
        ax = fig.add_subplot(gs[int(b / 5), int(b % 5)])
        ax.scatter(recon[b, :, 0], recon[b, :, 1], s=5)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_ylim(-bound, bound)
        ax.set_xlim(-bound, bound)
    
indices = torch.randperm(NUM_DATASETS)
batch_indices = indices[0*BATCH_SIZE : (0+1)*BATCH_SIZE]
ob = OB[batch_indices]
state = STATE[batch_indices]
mu = MU[batch_indices]
angle = ANGLE[batch_indices]


ob_embedding = torch.cat((ob, state, angle), -1)
ob_embedding = shuffler(ob_embedding).repeat(SAMPLE_SIZE, 1, 1, 1)
mu = mu.repeat(SAMPLE_SIZE, 1, 1, 1)
if CUDA:
    with torch.cuda.device(DEVICE):
        ob_embedding =ob_embedding.cuda()
        mu = mu.cuda()
ob = ob_embedding[:, :, :, :2]
state = ob_embedding[:, :, :, 2:-1]
angle = ob_embedding[:, :, :, -1].unsqueeze(-1)
p = dec_x(ob, state, angle, mu)

recon_mu = p['likelihood'].dist.loc[0].cpu().data.numpy()
plot_recon(recon_mu, PATH, 15, 10)

In [ ]:
plot_recon(ob[0].cpu().data.numpy(), PATH, 15, 10)